In [9]:
import requests
from bs4 import BeautifulSoup
import re
import json # テスト結果をきれいに出力するために使用

def extract_toshin_info(html_content, url):
    """答申詳細ページのHTMLから情報を抽出します（多様なフォーマット対応版）。"""
    soup = BeautifulSoup(html_content, 'html.parser')
    result = {
        'URL': url, '諮問庁': '', '諮問日': '', '答申日': '', '事件名': '',
        '第１_審査会の結論': '', '第２_審査請求人の主張の要旨': '', '第３_諮問庁の説明の要旨': '',
        '第４_参加人の主張の要旨': '',
        '第４_調査審議の経過': '', '第５_審査会の判断の理由': '', '委員': '', '別紙': ''
    }
    
    # === ヘッダー情報の抽出 (テーブル形式とPタグ形式に対応) ===
    header_table = soup.find('table')
    if header_table:
        rows = header_table.find_all('tr')
        # ヘッダーテーブルではない可能性を考慮し、行数が多すぎる場合は除外
        if len(rows) < 10:
            for row in rows:
                cells = [cell.get_text(strip=True) for cell in row.find_all('td')]
                if len(cells) < 2: continue
                
                label = cells[0].replace('：', '').strip()
                value_raw = cells[1]
                if len(cells) > 2 and cells[1] == '：':
                    value_raw = cells[2]
                value = value_raw.lstrip('：').strip()

                if '諮問庁' in label: result['諮問庁'] = value
                elif '諮問日' in label: result['諮問日'] = value
                elif '答申日' in label: result['答申日'] = value
                elif '事件名' in label: result['事件名'] = value

    if not soup.body: return result
    for tag in soup(["script", "style"]): tag.decompose()
    body_text = soup.body.get_text(separator='\n', strip=True)

    # Step 2: テーブルで取得できなかった場合、Pタグ形式を想定して正規表現で探す
    header_fields_map = {
        '諮問庁': r'^\s*諮問庁\s*[:：]\s*(.*)',
        '諮問日': r'^\s*諮問日\s*[:：]\s*(.*)',
        '答申日': r'^\s*答申日\s*[:：]\s*(.*)',
        '事件名': r'^\s*事件名\s*[:：]\s*(.*)',
    }
    for field, pattern in header_fields_map.items():
        if not result[field]:
            match = re.search(pattern, body_text, re.MULTILINE)
            if match:
                result[field] = match.group(1).strip()

    # === 本文セクションの抽出 (セクション番号のズレに対応) ===
    next_section_pattern = r'^\s*第\s*[\d０-９一二三四五六七八九十]+'
    sections = {
        '第１_審査会の結論': fr'^\s*第[１1]\s*[　\s]*(?:審査会の結論)?(.*?)(?={next_section_pattern}|\Z)',
        '第２_審査請求人の主張の要旨': fr'^\s*第\s*[\d０-９一二三四五六七八九十]+\s*[　\s]*(?:審査請求人|異議申立人)[の\s]+主張[の\s]+(?:要旨|概要)(.*?)(?={next_section_pattern}|\Z)',
        '第３_諮問庁の説明の要旨': fr'^\s*第\s*[\d０-９一二三四五六七八九十]+\s*[　\s]*諮問庁[の\s]+説明[の\s]+(?:要旨|概要)(.*?)(?={next_section_pattern}|\Z)',
        '第４_参加人の主張の要旨': fr'^\s*第\s*[\d０-９一二三四五六七八九十]+\s*[　\s]*参加人[の\s]+主張[の\s]+要旨(.*?)(?={next_section_pattern}|\Z)',
        '第４_調査審議の経過': fr'^\s*第\s*[\d０-９一二三四五六七八九十]+\s*[　\s]*調査審議[の\s]+経過(.*?)(?={next_section_pattern}|\Z)',
        '第５_審査会の判断の理由': fr'^\s*第\s*[\d０-９一二三四五六七八九十]+\s*[　\s]*審査会[の\s]+判断(?:の理由)?(.*?)(?=[（(]第\d+部会[）)]|{next_section_pattern}|^\s*第\s*[\d０-９一二三四五六七八九十]+\s*.*答申に関与した委員|\Z)'
    }
    
    for key, pattern in sections.items():
        match = re.search(pattern, body_text, re.DOTALL | re.MULTILINE)
        if match:
            content = match.group(1).strip()
            if content:
                result[key] = re.sub(r'\s*\n\s*', '\n', content)

    # === 委員・別紙の抽出 (複数パターンに対応する改善版) ===
    # 「第５」セクションより後のテキストを主な検索対象とする
    search_text = body_text
    dai5_match = re.search(sections['第５_審査会の判断の理由'], body_text, re.DOTALL | re.MULTILINE)
    if dai5_match:
        search_text = body_text[dai5_match.end():]

    # パターン1: （第X部会） パターン (例1, 2)
    committee_match = re.search(r'（第\d+部会）\s*(委員.+?)(?=\n\n|\n\s*別紙|\n\s*別表|\Z)', search_text, re.DOTALL)
    if committee_match:
        result['委員'] = committee_match.group(1).strip()
    else:
        # パターン2: 「第X 答申に関与した委員」 パターン (例3)
        committee_match = re.search(r'第\s*[\d０-９一二三四五六七八九十]+\s*.*答申に関与した委員\s*\n\s*(.+?)(?=\n\n|\n\s*別紙|\n\s*別表|\Z)', search_text, re.DOTALL)
        if committee_match:
            result['委員'] = committee_match.group(1).strip()
    
    # どのパターンでも見つからなかった場合、body_text全体から最終手段で探す
    if not result['委員']:
        committee_match = re.search(r'^\s*委員\s*([^委員\n]+(?:、委員\s*[^委員\n]+)*)', body_text, re.MULTILINE)
        if committee_match:
            result['委員'] = committee_match.group(0).strip()

    # 別紙の抽出
    besshi_match = re.search(r'(\n\s*(?:別紙|別表).*)', body_text, re.DOTALL)
    if besshi_match:
        result['別紙'] = besshi_match.group(1).strip()
        
    return result


In [15]:
test_url = 'https://koukai-hogo-db.soumu.go.jp/reportBody/1923'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

print(f"--- URLのテストを開始 ---")
print(f"対象URL: {test_url}\n")

try:
    # URLからHTMLコンテンツを取得
    response = requests.get(test_url, headers=headers, timeout=20)
    response.raise_for_status() # ステータスコードが200以外ならエラーを発生させる
    response.encoding = response.apparent_encoding # 文字化け防止

    # 関数を実行して情報を抽出
    extracted_data = extract_toshin_info(response.text, test_url)

    # 結果を整形して表示
    print("\n--- 抽出結果 ---")
    print(json.dumps(extracted_data, ensure_ascii=False, indent=2))

except requests.RequestException as e:
    print(f"エラー: URLの取得に失敗しました。 {e}")


--- URLのテストを開始 ---
対象URL: https://koukai-hogo-db.soumu.go.jp/reportBody/1923


--- 抽出結果 ---
{
  "URL": "https://koukai-hogo-db.soumu.go.jp/reportBody/1923",
  "諮問庁": "外務大臣",
  "諮問日": "平成１７年　１月　５日　（平成１７年（行情）諮問第３号）",
  "答申日": "平成１７年　３月２９日　（平成１６年度（行情）答申第６５５号）",
  "事件名": "「対米武器技術供与に関する基本方針」等の不開示決定に関する件",
  "第１_審査会の結論": "別表のⅠ欄に掲げる文書を不開示とした決定については，別表のⅡ欄に掲げる部分を開示すべきである。",
  "第２_審査請求人の主張の要旨": "１\n異議申立ての趣旨\n本件異議申立ての趣旨は，行政機関の保有する情報の公開に関する法律（以下「法」という。）３条の規定に基づく本件対象文書の開示請求に対し，平成１６年７月６日付け情報公開第０２５２７号により外務大臣が行った不開示決定（以下「本件決定」という。）について，その取消しを求めるというものである。\n２\n異議申立ての理由\n異議申立書の記載によると，異議申立人の主張はおおむね以下のとおりである。\n（１）\n対米武器技術供与に関する当時の外務省の考え方や立場，問題提起，その後の通産，外務，防衛の三省庁会議で「相互防衛援助協定などの存在により，法制面では対米供与はできる」という合意がされた等の点は膨大な新聞報道など既に公になっている。したがって，これらの情報が開示されたからといって，直ちに，不開示決定通知書が挙げる事由のおそれが生じるわけではない。\n（２）\n対米武器技術供与については，昭和５８年１月の官房長官談話以降，具体的な技術交流，供与などが２０年以上にわたって積み重ねられており，この問題をめぐる議論が官房長官談話以前の地点に戻るとは考えられない。また，対象は米国のみ，中身も技術と限定的であって，現在の武器技術（輸出）三原則をめぐる問題とは次元が違う問題である。したがって，これらの情報が開示されたからといって，不開示決定通知書が挙げる事由のおそれが生じるわけではない。",
  "第３

In [8]:
response = requests.post("https://koukai-hogo-db.soumu.go.jp/reportBody/1923")
soup = BeautifulSoup(response.text, 'html.parser')
soup


<html>
<head>
<title>答申本文</title>
<link href="/images/favicon.ico;jsessionid=AA7AB56E819CBA55197854BA812ABFC8" rel="shortcut icon" type="image/x-icon"/>
</head>
<body>
<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<title>答申本文</title>
<style type="text/css">
	<!--
		BODY{font-size:10pt; color=black}
		TABLE{font-size:10pt; color=black}
	-->
</style>
</head>
<body>
<table>
<tr align="left" valign="top"><td nowrap="">諮問庁</td><td>：</td><td>外務大臣</td></tr>
<tr align="left" valign="top"><td nowrap="">諮問日</td><td>：</td><td>平成１７年　１月　５日　（平成１７年（行情）諮問第３号）</td></tr>
<tr align="left" valign="top"><td nowrap="">答申日</td><td>：</td><td>平成１７年　３月２９日　（平成１６年度（行情）答申第６５５号）</td></tr>
<tr align="left" valign="top"><td nowrap="">事件名</td><td>：</td><td>「対米武器技術供与に関する基本方針」等の不開示決定に関する件</td></tr>
</table>
<br/>
<center><font size="5"><b>答　申　書</b></font><br/></center>
<br/>
<div style="margin-left:15pt">
<table>
<tr align="left" valign="top">
<!--1//-->
<td nowrap="">第１</td>
<td>　審

In [18]:
import requests

# フォームデータを送信する先のURL
url = "https://koukai-hogo-db.soumu.go.jp/report/search"

# 送信するデータ（ペイロード）を定義
# オンにしたいチェックボックスのname属性をキーに、値を'on'に設定
payload = {
    'gyouseiKoukai': 'on',
    'gyouseiHogo': 'on',
    'dokuhouKoukai': 'on',
    'dokuhouHogo': 'on',
    'openFlag': '1',  # HTML内にあった隠しフィールドも送信
    'status': '',     # 同上
    # 他の検索条件（キーワードや日付など）は空のまま送信
    'searchQuery': '',
    'consultDateFromEra': '',
    'consultDateFromYear': '',
    'consultDateFromMonth': '',
    'consultDateFromDate': '',
    # ... その他のフォーム項目も必要に応じて追加 ...
}

try:
    # POSTリクエストを送信
    response = requests.post(url, data=payload)

    # ステータスコードが200（成功）か確認
    response.raise_for_status() 

    # 検索結果ページのHTMLを取得
    search_results_html = response.text
    
    print("リクエストは成功しました。")
    print(f"ステータスコード: {response.status_code}")
    
    # ここでsearch_results_htmlをBeautifulSoupなどのライブラリで解析し、
    # 必要な情報を抽出することができます。
    # print(search_results_html[:1000]) # 結果のHTMLの先頭1000文字を表示

except requests.exceptions.RequestException as e:
    print(f"リクエスト中にエラーが発生しました: {e}")

リクエストは成功しました。
ステータスコード: 200


In [1]:
import time
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# --- Seleniumのセットアップ ---
print("Selenium WebDriverをセットアップしています...")
chrome_options = Options()
chrome_options.add_argument("--headless") # ブラウザを画面に表示せずに実行する場合
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
print("WebDriverのセットアップ完了。")

try:
    # --- 検索ページの操作 ---
    print("検索ページにアクセスし、条件を設定します...")
    driver.get('https://koukai-hogo-db.soumu.go.jp/report/')
    time.sleep(2) # ページが完全に読み込まれるのを待つ

    # 4つのチェックボックスをすべてクリック
    checkbox_names = ['gyouseiKoukai', 'gyouseiHogo', 'dokuhouKoukai', 'dokuhouHogo']
    for name in checkbox_names:
        driver.find_element(By.NAME, name).click()
        
    # 「検索」ボタンをクリック
    search_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='検索']")
    search_button.click()
    
    print("検索を実行しました。結果ページを解析します...")
    time.sleep(2) # 検索結果が表示されるのを待つ

    # --- 総件数の取得 ---
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # "1～20 ／ 22,167件を表示しています。" というテキスト部分を取得
    paging_item_text = soup.select_one('div.paging-item').get_text(strip=True)
    
    # テキストから件数部分の数字だけを抽出
    total_items_str = paging_item_text.split('／')[1] # "／"で分割して後半部分を取得
    total_items = int("".join(filter(str.isdigit, total_items_str)))
    
    print("\n----------------------------------------")
    print(f"取得した総件数: {total_items}")
    print("----------------------------------------")

    if total_items == 22167:
        print("✅ 正常に総件数(22,167件)を取得できました！")
    else:
        print(f"⚠️ 取得した件数が想定と異なります: {total_items}件")

except Exception as e:
    print(f"処理中にエラーが発生しました: {e}")
    
finally:
    # --- ブラウザを閉じる ---
    print("ブラウザを閉じます。")
    driver.quit()

Selenium WebDriverをセットアップしています...
WebDriverのセットアップ完了。
検索ページにアクセスし、条件を設定します...
検索を実行しました。結果ページを解析します...

----------------------------------------
取得した総件数: 6514
----------------------------------------
⚠️ 取得した件数が想定と異なります: 6514件
ブラウザを閉じます。


In [2]:
# %% [markdown]
# # 総務省答申データベース検索の確認
# 4つの答申種別すべてにチェックを入れて、22,167件の検索結果を取得できるか確認

# %%
import requests
from bs4 import BeautifulSoup
import re

# %%
# セッションの作成
session = requests.Session()

# ヘッダーの設定
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'ja,en-US;q=0.7,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Origin': 'https://koukai-hogo-db.soumu.go.jp',
    'Referer': 'https://koukai-hogo-db.soumu.go.jp/report/'
}

# %%
# まず検索ページにアクセス（クッキー取得）
print("1. 検索ページにアクセス...")
response_get = session.get('https://koukai-hogo-db.soumu.go.jp/report/', headers=headers)
print(f"   ステータスコード: {response_get.status_code}")
print(f"   クッキー: {session.cookies.get_dict()}")

# %%
# フォームデータの準備（4つすべてチェック）
form_data = {
    # 答申種別 - 4つすべてにチェック
    'gyouseiKoukai': 'on',    # 行政機関／情報公開法
    'gyouseiHogo': 'on',      # 行政機関／個人情報保護法  
    'dokuhouKoukai': 'on',     # 独立行政法人等／情報公開法
    'dokuhouHogo': 'on',       # 独立行政法人等／個人情報保護法
    
    # 請求種別（個人情報保護法用）- チェックなし
    # 'kaijiSeikyu': '',
    # 'teiseiSeikyu': '',
    # 'teishiSeikyu': '',
    
    # 用語検索
    'searchType': '1',         # 検索タイプ
    'searchQuery': '',         # 検索語句（空）
    
    # 日付関連（すべて空）
    'consultDateFromEra': '',
    'consultDateFromYear': '',
    'consultDateFromMonth': '',
    'consultDateFromDate': '',
    'consultDateRange': '',
    'consultDateToEra': '',
    'consultDateToYear': '',
    'consultDateToMonth': '',
    'consultDateToDate': '',
    
    # 諮問番号（すべて空）
    'consultNoFromEra': '',
    'consultNoFromYear': '',
    'consultNoFromNo': '',
    'consultNoRange': '',
    'consultNoToEra': '',
    'consultNoToYear': '',
    'consultNoToNo': '',
    
    # 諮問庁（空）
    'consultAgency': '',
    'consultAgencyQuery': '',
    
    # 答申日（すべて空）
    'reportDateFromEra': '',
    'reportDateFromYear': '',
    'reportDateFromMonth': '',
    'reportDateFromDate': '',
    'reportDateRange': '',
    'reportDateToEra': '',
    'reportDateToYear': '',
    'reportDateToMonth': '',
    'reportDateToDate': '',
    
    # 答申番号（すべて空）
    'reportNoFromEra': '',
    'reportNoFromYear': '',
    'reportNoFromNo': '',
    'reportNoRange': '',
    'reportNoToEra': '',
    'reportNoToYear': '',
    'reportNoToNo': '',
    
    # 諮問事件名（空）
    'consultAffairName': '',
    
    # 表示順序
    'order1': '12',           # 答申日の新しい順
    'order2': '',
    'order3': '',
    'limitOption': '1',       # 20件ずつ
    
    # 必須フラグ
    'openFlag': '1',
    'status': ''
}

print("\n2. フォームデータを準備しました")
print(f"   チェックした答申種別: 行情、独情、行個、独個の4つすべて")

# %%
# 検索実行
print("\n3. 検索を実行...")
search_url = 'https://koukai-hogo-db.soumu.go.jp/report/search'
response = session.post(search_url, data=form_data, headers=headers, allow_redirects=True)

print(f"   ステータスコード: {response.status_code}")
print(f"   レスポンスURL: {response.url}")
print(f"   レスポンスサイズ: {len(response.text)} bytes")

# %%
# 検索結果の解析
soup = BeautifulSoup(response.text, 'html.parser')

# 検索結果件数を探す - 複数のパターンで探索
print("\n4. 検索結果の解析...")

# パターン1: "検索結果 XXXX件" のような表示を探す
result_text = soup.get_text()
match = re.search(r'検索結果[：:\s]*(\d+[,\d]*)\s*件', result_text)
if match:
    count = match.group(1).replace(',', '')
    print(f"   検索結果件数: {count}件")

# パターン2: "全XXXX件" のような表示を探す
match2 = re.search(r'全\s*(\d+[,\d]*)\s*件', result_text)
if match2:
    count = match2.group(1).replace(',', '')
    print(f"   全件数: {count}件")

# パターン3: ヒット件数が含まれるdivやspanを探す
hit_count_elements = soup.find_all(['div', 'span'], string=re.compile(r'\d+[,\d]*\s*件'))
if hit_count_elements:
    print("   件数表示が見つかりました:")
    for elem in hit_count_elements[:3]:  # 最初の3つだけ表示
        print(f"     - {elem.get_text().strip()}")

# %%
# 検索結果のテーブルを確認
tables = soup.find_all('table')
print(f"\n5. テーブル要素: {len(tables)}個見つかりました")

# 結果テーブルを探す
for i, table in enumerate(tables):
    # class属性を確認
    table_class = table.get('class', [])
    if table_class:
        print(f"   テーブル{i+1}: class={table_class}")
    
    # 結果らしきテーブルを探す
    rows = table.find_all('tr')
    if len(rows) > 1:  # ヘッダー以外に行がある
        first_row = rows[1] if len(rows) > 1 else rows[0]
        cells = first_row.find_all(['td', 'th'])
        if len(cells) > 3:  # 複数カラムがある
            print(f"   テーブル{i+1}に{len(rows)}行のデータがあります")
            # 最初の行の内容を表示
            print(f"   最初の行: {' | '.join([cell.get_text().strip()[:20] for cell in cells[:4]])}")

# %%
# HTMLを保存して詳細確認
with open('search_result_check.html', 'w', encoding='utf-8') as f:
    f.write(response.text)
print("\n6. 検索結果を 'search_result_check.html' に保存しました")
print("   ブラウザで開いて内容を確認してください")

# %%
# エラーメッセージや警告を確認
error_messages = soup.find_all(['div', 'span'], class_=re.compile(r'error|alert|warning', re.I))
if error_messages:
    print("\n⚠️ エラーまたは警告メッセージが見つかりました:")
    for msg in error_messages:
        print(f"   {msg.get_text().strip()}")

# 特定のメッセージパターンを探す
if "検索条件を指定してください" in result_text:
    print("\n⚠️ 検索条件の指定が必要というメッセージが表示されています")
if "該当するデータがありません" in result_text:
    print("\n⚠️ 該当データなしのメッセージが表示されています")
if "エラーが発生しました" in result_text:
    print("\n⚠️ エラーメッセージが表示されています")

# %%
# デバッグ用: レスポンスの最初の部分を表示
print("\n7. レスポンスHTMLの最初の1000文字:")
print(response.text[:1000])

# %%
# フォームの送信先とメソッドを確認（検索ページから）
search_page_soup = BeautifulSoup(response_get.text, 'html.parser')
form = search_page_soup.find('form', {'name': 'reportActionForm'})
if form:
    print("\n8. 元のフォーム情報:")
    print(f"   action: {form.get('action')}")
    print(f"   method: {form.get('method')}")
    
    # チェックボックスの name 属性を確認
    checkboxes = form.find_all('input', {'type': 'checkbox'})
    print(f"\n   チェックボックス一覧:")
    for cb in checkboxes:
        print(f"     - name='{cb.get('name')}', value='{cb.get('value')}', checked={cb.has_attr('checked')}")

# %%
print("\n=== 確認完了 ===")
print("HTMLファイルを確認し、実際の検索結果が表示されているか確認してください。")
print("もし22,167件の結果が表示されていない場合は、")
print("HTMLの内容を確認して、追加のパラメータが必要か確認する必要があります。")

1. 検索ページにアクセス...
   ステータスコード: 200
   クッキー: {'AWSALB': '7HB+b1GLOUDCSp16WnswLu+XIF8VNQrXABmlSnYpx05hjPPasaQegJ7C9VoVdEW3D5oxqgXHVaEhVWwfpghDX8fC2Bc8kqb4YRMj4gDOCP86/NdBiNjuHNSpZvre', 'AWSALBCORS': '7HB+b1GLOUDCSp16WnswLu+XIF8VNQrXABmlSnYpx05hjPPasaQegJ7C9VoVdEW3D5oxqgXHVaEhVWwfpghDX8fC2Bc8kqb4YRMj4gDOCP86/NdBiNjuHNSpZvre', 'JSESSIONID': 'DA1BD3063E09735B6E779CD1F17DB666'}

2. フォームデータを準備しました
   チェックした答申種別: 行情、独情、行個、独個の4つすべて

3. 検索を実行...
   ステータスコード: 200
   レスポンスURL: https://koukai-hogo-db.soumu.go.jp/report/search
   レスポンスサイズ: 45043 bytes

4. 検索結果の解析...

5. テーブル要素: 42個見つかりました
   テーブル1: class=['search-result']
   テーブル2: class=['search-result']
   テーブル3: class=['search-result']
   テーブル4: class=['search-result']
   テーブル5: class=['search-result']
   テーブル6: class=['search-result']
   テーブル7: class=['search-result']
   テーブル8: class=['search-result']
   テーブル9: class=['search-result']
   テーブル10: class=['search-result']
   テーブル11: class=['search-result']
   テーブル12: class=['search-result']
   テーブル1

In [6]:
# %% [markdown]
# # 総務省答申データベース検索 - 60件取得版

# %%
import requests
from bs4 import BeautifulSoup
import re
import csv

# %%
# セッションの作成とヘッダー設定
session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Referer': 'https://koukai-hogo-db.soumu.go.jp/report/'
}

# %%
# 検索ページにアクセス
print("検索ページにアクセス...")
session.get('https://koukai-hogo-db.soumu.go.jp/report/', headers=headers)

# %%
# フォームデータ（4つすべてチェック）
form_data = {
    'gyouseiKoukai': 'on',    # 行政機関／情報公開法
    'gyouseiHogo': 'on',      # 行政機関／個人情報保護法  
    'dokuhouKoukai': 'on',     # 独立行政法人等／情報公開法
    'dokuhouHogo': 'on',       # 独立行政法人等／個人情報保護法
    'searchType': '1',
    'searchQuery': '',
    'order1': '12',
    'limitOption': '1',
    'openFlag': '1',
    'status': ''
}

# %%
# 初回検索実行
print("検索を実行...")
search_url = 'https://koukai-hogo-db.soumu.go.jp/report/search'
response = session.post(search_url, data=form_data, headers=headers, allow_redirects=True)
print(f"ステータスコード: {response.status_code}")


検索ページにアクセス...
検索を実行...
ステータスコード: 200


In [7]:
# %% [markdown]
# ## 検索結果251ページ目の情報を抽出

# %%
# 抽出したいページ番号を指定
target_page_num = 251
target_url = f"https://koukai-hogo-db.soumu.go.jp/report/search/page/{target_page_num}"

print(f"{target_page_num}ページ目のHTMLを取得します...")
print(f"URL: {target_url}")

# ページにアクセスしてHTMLを取得
try:
    # 設定済みのセッションとヘッダーを使用してページを取得
    page_response = session.get(target_url, headers=headers, timeout=15)
    page_response.raise_for_status()  # ステータスコードが200番台でない場合にエラーを発生させる

    print("ページの取得に成功しました。")

    # --- BeautifulSoupオブジェクトを作成してHTMLをパース ---
    soup = BeautifulSoup(page_response.text, 'html.parser')
    print("HTMLのパースが完了しました。変数'soup'に格納されています。")

    # 例として、soupオブジェクトからページのタイトルを表示
    if soup.title:
        print(f"ページのタイトル: {soup.title.string}")

    # --- 既存の関数を使って情報を抽出 ---
    print("\n既存のextract_report_info関数を使って情報を抽出します...")
    results_page_251 = extract_report_info(page_response.text)

    # 抽出結果の確認
    if results_page_251:
        print(f"=>{len(results_page_251)}件の情報を抽出しました。")

        # 抽出した情報の最初の1件を表示して確認
        print("\n--- 抽出結果（最初の1件）---")
        # jsonをインポートしてきれいに表示
        import json
        print(json.dumps(results_page_251[0], ensure_ascii=False, indent=2))
    else:
        print("このページから情報を抽出できませんでした。")

except requests.exceptions.RequestException as e:
    print(f"リクエスト中にエラーが発生しました: {e}")
except Exception as e:
    print(f"予期せぬエラーが発生しました: {e}")

251ページ目のHTMLを取得します...
URL: https://koukai-hogo-db.soumu.go.jp/report/search/page/251
ページの取得に成功しました。
HTMLのパースが完了しました。変数'soup'に格納されています。
ページのタイトル: 答申データベース検索

既存のextract_report_info関数を使って情報を抽出します...
予期せぬエラーが発生しました: name 'extract_report_info' is not defined


In [8]:
soup


<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<title>答申データベース検索</title>
<link href="/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/stylesheets/common.css" media="screen" rel="stylesheet"/>
<link href="/bootstrap/css/bootstrap.css" media="screen" rel="stylesheet"/>
<link href="/bootstrap/css/bootstrap.form.css" media="screen" rel="stylesheet"/>
<link href="/treeview/jquery.treeview.css" media="screen" rel="stylesheet"/>
<script src="/javascripts/common.js" type="text/javascript"></script>
<script src="/javascripts/jquery-1.8.0.min.js" type="text/javascript"></script>
<script src="/javascripts/jquery.cookie.js" type="text/javascript"></script>
<script src="/bootstrap/js/bootstrap-alert.js" type="text/javascript"></script>
<script src="/treeview/jquery.treeview.js" type="text/javascript"></script>
<script src="/treeview/jquery.treeview.async.js" type="text/javascript"></script>
</head>
<body data-spy="scroll">
<div class="

In [8]:
# %% [markdown]
# ## 検索結果から「本文表示」のURLを取得（3ページ分・60件）

# %%
def extract_report_info(html_content):
    """HTMLから本文表示のURLと詳細情報を抽出"""
    soup = BeautifulSoup(html_content, 'html.parser')
    results = []
    
    for table in soup.find_all('table', class_='search-result'):
        record = {}
        
        # 本文表示リンクを取得
        link = table.find('a', class_='btn btn-mini')
        if link and link.get('href'):
            href = link.get('href')
            record['本文URL'] = f"https://koukai-hogo-db.soumu.go.jp{href}" if href.startswith('/') else href
        
        # 各行から情報を抽出
        for row in table.find_all('tr', class_='search-result-body'):
            cells = row.find_all('td')
            if len(cells) >= 2:
                label = cells[0].get_text(strip=True)
                value = cells[1].get_text(strip=True)
                
                if '答申日／答申番号' in label:
                    record['答申日'] = value
                elif '諮問日／諮問番号' in label:
                    record['諮問日'] = value
                elif '諮問庁' in label:
                    record['諮問庁'] = value
                elif '事件名' in label:
                    record['事件名'] = value
        
        if record:
            results.append(record)
    
    return results

# %%
def get_search_results(session, response, headers, num_pages=3):
    """検索結果から指定ページ数分のデータを取得"""
    all_results = []
    all_urls = []

    for page_num in range(1, num_pages + 1):
        print(f"{page_num}ページ目を取得中...")
        
        if page_num == 1:
            page_html = response.text
        else:
            page_response = session.get(f"https://koukai-hogo-db.soumu.go.jp/report/search/page/{page_num}", headers=headers)
            if page_response.status_code != 200:
                print(f"  エラー: ステータスコード {page_response.status_code}")
                continue
            page_html = page_response.text
        
        # URLを抽出
        soup = BeautifulSoup(page_html, 'html.parser')
        for link in soup.find_all('a', class_='btn btn-mini'):
            href = link.get('href')
            if href:
                all_urls.append(f"https://koukai-hogo-db.soumu.go.jp{href}" if href.startswith('/') else href)
        
        # 詳細情報を抽出
        page_results = extract_report_info(page_html)
        all_results.extend(page_results)
        print(f"  {len(page_results)}件取得")
    
    return all_urls, all_results

# %%
# 3ページ分のデータを取得
all_urls, all_results = get_search_results(session, response, headers, num_pages=3)

print(f"\n=== 取得完了 ===")
print(f"URL: {len(all_urls)}件")
print(f"詳細情報: {len(all_results)}件")

# %%
import json

# JSONで保存
with open('soumu_search_results_60.json', 'w', encoding='utf-8') as f:
    json.dump(all_results, f, ensure_ascii=False, indent=2)

# URLリストを保存
with open('report_urls_60.txt', 'w', encoding='utf-8') as f:
    for url in all_urls:
        f.write(url + '\n')

print("\n保存完了")

1ページ目を取得中...
  20件取得
2ページ目を取得中...
  20件取得
3ページ目を取得中...
  20件取得

=== 取得完了 ===
URL: 60件
詳細情報: 60件

保存完了


In [ ]:
import re
from bs4 import BeautifulSoup

def extract_toshin_info(html_content, url):
    """HTMLから答申情報を抽出（改善版 v4 - 3つ以上の改行を1つに集約）"""
    soup = BeautifulSoup(html_content, 'html.parser')
    
    result = {
        'URL': url,
        '諮問庁': '', '諮問日': '', '諮問番号': '', '答申日': '', '答申番号': '', '事件名': '',
        '第１_審査会の結論': '', '第２_審査請求人の主張の要旨': '', '第３_諮問庁の説明の要旨': '',
        '第４_調査審議の経過': '', '第５_審査会の判断の理由': '', '委員': '', '別紙': ''
    }
    
    # ヘッダー情報の抽出
    header_table = soup.find('table')
    if header_table:
        for row in header_table.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) >= 3:
                label = cells[0].get_text(strip=True)
                value = cells[2].get_text(strip=True)
                
                if '諮問庁' in label:
                    result['諮問庁'] = value
                elif '諮問日' in label:
                    result['諮問日'] = value
                    match = re.search(r'（([^）]+)）', value)
                    if match:
                        result['諮問番号'] = match.group(1)
                elif '答申日' in label:
                    result['答申日'] = value
                    match = re.search(r'（([^）]+)）', value)
                    if match:
                        result['答申番号'] = match.group(1)
                elif '事件名' in label:
                    result['事件名'] = value
    
    body_div = soup.find('div', style="font-family:'ＭＳ ゴシック';")
    if body_div:
        full_text = body_div.get_text(separator='\n')
        
        # 本文セクションの抽出
        sections = {
            '第１_審査会の結論': r'第１\s*審査会の結論(.*?)(?=^\s*第２|\Z)',
            '第２_審査請求人の主張の要旨': r'第２\s*審査請求人の主張の要旨(.*?)(?=^\s*第３|\Z)',
            '第３_諮問庁の説明の要旨': r'第３\s*諮問庁の説明の要旨(.*?)(?=^\s*第４|\Z)',
            '第４_調査審議の経過': r'第４\s*調査審議の経過(.*?)(?=^\s*第５|\Z)',
            '第５_審査会の判断の理由': r'第５\s*審査会の判断の理由(.*?)(?=^\s*（第\d部会）|^\s*委員|^\s*別紙|\Z)'
        }
        
        for key, pattern in sections.items():
            match = re.search(pattern, full_text, re.DOTALL | re.MULTILINE)
            if match:
                raw_text = match.group(1).strip()
                cleaned_text = re.sub(r'\n{3,}', '\n', raw_text)
                result[key] = cleaned_text
        
        # 委員名の抽出
        committee_tag = soup.find('p', string=re.compile(r'^\s*委員'))
        if committee_tag:
            result['委員'] = committee_tag.get_text(strip=True)

        # 「別紙」の抽出
        besshi_match = re.search(r'(^\s*別紙\s*$)', full_text, re.MULTILINE)
        if besshi_match:
            start_index = besshi_match.start()
            raw_text = full_text[start_index:].strip()
            cleaned_text = re.sub(r'\n{3,}', '\n', raw_text)
            result['別紙'] = cleaned_text
            
    return result

# %%
def extract_all_toshin_data(urls, session, headers, sleep_time=1):
    """URLリストから答申データを抽出"""
    extracted_data = []
    failed_urls = []
    
    print(f"答申本文の取得を開始... 全{len(urls)}件")
    
    for i, url in enumerate(urls, 1):
        if i % 10 == 0:
            print(f"  {i}/{len(urls)}件処理済")
        
        try:
            if i > 1:
                time.sleep(sleep_time)
            
            response = session.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                info = extract_toshin_info(response.text, url)
                extracted_data.append(info)
            else:
                failed_urls.append(url)
        except Exception:
            failed_urls.append(url)
    
    print(f"\n完了: 成功 {len(extracted_data)}件, 失敗 {len(failed_urls)}件")
    return extracted_data, failed_urls

# %%
# 60件のURLから答申データを抽出
toshin_data, failed = extract_all_toshin_data(all_urls, session, headers)

# %%
# 保存
with open('toshin_data_60.json', 'w', encoding='utf-8') as f:
    json.dump(toshin_data, f, ensure_ascii=False, indent=2)

fieldnames = ['URL', '諮問庁', '諮問日', '諮問番号', '答申日', '答申番号', '事件名',
              '第１_審査会の結論', '第２_審査請求人の主張の要旨', '第３_諮問庁の説明の要旨',
              '第４_調査審議の経過', '第５_審査会の判断の理由', '委員', '別紙']

with open('toshin_data_60.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(toshin_data)

print("保存完了: toshin_data_60.json, toshin_data_60.csv")

答申本文の取得を開始... 全60件
  10/60件処理済
  20/60件処理済
  30/60件処理済
  40/60件処理済
  50/60件処理済
  60/60件処理済

完了: 成功 60件, 失敗 0件
保存完了: toshin_data_60.json, toshin_data_60.csv


In [10]:
response = requests.post("https://koukai-hogo-db.soumu.go.jp/reportBody/17778")
soup = BeautifulSoup(response.text, 'html.parser')
soup


<html>
<head>
<title>答申本文</title>
<link href="/images/favicon.ico;jsessionid=F49ABFFE7BAC334FABBAD2D7AC1AB6F7" rel="shortcut icon" type="image/x-icon"/>
</head>
<body>
				
				　    <div>
<table>
<tbody>
<tr align="left" valign="top">
<td nowrap="nowrap">諮問庁</td>
<td>：</td>
<td>厚生労働大臣</td>
</tr>
<tr align="left" valign="top">
<td nowrap="nowrap">諮問日</td>
<td>：</td>
<td>令和３年６月２９日（令和３年（行情）諮問第２７２号）</td>
</tr>
<tr align="left" valign="top">
<td nowrap="nowrap">答申日</td>
<td>：</td>
<td>令和６年４月１２日（令和６年度（行情）答申第７号）</td>
</tr>
<tr align="left" valign="top">
<td nowrap="nowrap">事件名</td>
<td>：</td>
<td>「ワクチン等の品質確保を目的とした新たな国家検定システムの構築のための研究」における記述の根拠となった諸外国と日本の検定実施期間を比較した文書の不開示決定（不存在）に関する件</td>
</tr>
</tbody>
</table>
<p style="text-align:justify;margin:0;font-family:'ＭＳ ゴシック';">
<br style="font-family:'ＭＳ ゴシック';"/>
</p>
<p style="text-align:justify;margin:0;font-family:'ＭＳ ゴシック';">
<br style="font-family:'ＭＳ ゴシック';"/>
</p>
<p style="text-align:center;font-size:18pt;margin:0;font-family:'ＭＳ ゴシック';f

In [12]:
response = requests.post("https://koukai-hogo-db.soumu.go.jp/reportBody/10")
soup = BeautifulSoup(response.text, 'html.parser')
soup


<html>
<head>
<title>答申本文</title>
<link href="/images/favicon.ico;jsessionid=BFEE1820E0CCB11B8E889D72C29E7591" rel="shortcut icon" type="image/x-icon"/>
</head>
<body>
<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<title>答申本文</title>
<style type="text/css">
	<!--
		BODY{font-size:10pt; color=black}
		TABLE{font-size:10pt; color=black}
	-->
</style>
</head>
<body>
<table>
<tr align="left" valign="top"><td nowrap="">諮問庁</td><td>：検事総長</td></tr>
<tr align="left" valign="top"><td nowrap="">諮問日</td><td>：平成１３年７月２３日</td></tr>
<tr align="left" valign="top"><td nowrap="">答申日</td><td>：平成１３年１０月２日</td></tr>
<tr align="left" valign="top"><td nowrap="">事件名</td><td>：明治３９年検務事件簿中の特定個人に係る記載部分の不開示決定（存否応答拒否）に関する件(平成１３年諮問第３５号)</td></tr>
</table>
<br/>
<center><font size="5"><b>答　申　書</b></font><br/></center>
<br/>
<div style="margin-left:17pt">
<table><tr align="left" valign="top">
<!--1//-->
<td nowrap="">第１</td><td>審査会の結論<br/></td></tr></table>
<div style="margin-left:

In [ ]:
import time  # これを追加

# %%
def extract_toshin_info(html_content, url):
    """HTMLから答申情報を抽出"""
    soup = BeautifulSoup(html_content, 'html.parser')
    
    result = {
        'URL': url,
        '諮問庁': '',
        '諮問日': '',
        '諮問番号': '',
        '答申日': '',
        '答申番号': '',
        '事件名': '',
        '第１_審査会の結論': '',
        '第２_審査請求人の主張の要旨': '',
        '第３_諮問庁の説明の要旨': '',
        '第４_調査審議の経過': '',
        '第５_審査会の判断の理由': '',
        '委員': '',
        '別紙': ''
    }
    
    # ヘッダー情報
    header_table = soup.find('table')
    if header_table:
        for row in header_table.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) >= 3:
                label = cells[0].get_text(strip=True)
                value = cells[2].get_text(strip=True)
                
                if '諮問庁' in label:
                    result['諮問庁'] = value
                elif '諮問日' in label:
                    result['諮問日'] = value
                    match = re.search(r'（([^）]+)）', value)
                    if match:
                        result['諮問番号'] = match.group(1)
                elif '答申日' in label:
                    result['答申日'] = value
                    match = re.search(r'（([^）]+)）', value)
                    if match:
                        result['答申番号'] = match.group(1)
                elif '事件名' in label:
                    result['事件名'] = value
    
    # 本文セクション
    full_text = soup.get_text()
    sections = {
        '第１_審査会の結論': r'第１\s*審査会の結論(.*?)(?=第２|$)',
        '第２_審査請求人の主張の要旨': r'第２\s*審査請求人の主張の要旨(.*?)(?=第３|$)',
        '第３_諮問庁の説明の要旨': r'第３\s*諮問庁の説明の要旨(.*?)(?=第４|$)',
        '第４_調査審議の経過': r'第４\s*調査審議の経過(.*?)(?=第５|$)',
        '第５_審査会の判断の理由': r'第５\s*審査会の判断の理由(.*?)(?=（第\d部会）|委員|別紙|$)'
    }
    
    for key, pattern in sections.items():
        match = re.search(pattern, full_text, re.DOTALL)
        if match:
            text = re.sub(r'\s+', ' ', match.group(1).strip())
            result[key] = text[:500]
    
    # 委員名
    committee_match = re.search(r'委員\s+([^、]+)、委員\s+([^、]+)、委員\s+([^\s]+)', full_text)
    if committee_match:
        result['委員'] = '、'.join(committee_match.groups())
    
    # 別紙
    bessi_match = re.search(r'別紙(.*?)$', full_text, re.DOTALL)
    if bessi_match:
        text = re.sub(r'\s+', ' ', bessi_match.group(1).strip())
        result['別紙'] = text[:500]
    
    return result

# %%
def extract_all_toshin_data(urls, session, headers, sleep_time=1):
    """URLリストから答申データを抽出"""
    extracted_data = []
    failed_urls = []
    
    print(f"答申本文の取得を開始... 全{len(urls)}件")
    
    for i, url in enumerate(urls, 1):
        if i % 10 == 0:
            print(f"  {i}/{len(urls)}件処理済")
        
        try:
            if i > 1:
                time.sleep(sleep_time)
            
            response = session.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                info = extract_toshin_info(response.text, url)
                extracted_data.append(info)
            else:
                failed_urls.append(url)
        except Exception:
            failed_urls.append(url)
    
    print(f"\n完了: 成功 {len(extracted_data)}件, 失敗 {len(failed_urls)}件")
    return extracted_data, failed_urls

# %%
# 60件のURLから答申データを抽出
toshin_data, failed = extract_all_toshin_data(all_urls, session, headers)

# %%
# 保存
with open('toshin_data_60.json', 'w', encoding='utf-8') as f:
    json.dump(toshin_data, f, ensure_ascii=False, indent=2)

fieldnames = ['URL', '諮問庁', '諮問日', '諮問番号', '答申日', '答申番号', '事件名',
              '第１_審査会の結論', '第２_審査請求人の主張の要旨', '第３_諮問庁の説明の要旨',
              '第４_調査審議の経過', '第５_審査会の判断の理由', '委員', '別紙']

with open('toshin_data_60.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(toshin_data)

print("保存完了: toshin_data_60.json, toshin_data_60.csv")

答申本文の取得を開始... 全60件
  10/60件処理済
  20/60件処理済
  30/60件処理済
  40/60件処理済
  50/60件処理済
  60/60件処理済

完了: 成功 60件, 失敗 0件
保存完了: toshin_data_60.json, toshin_data_60.csv


'https://koukai-hogo-db.soumu.go.jp/reportBody/19170'

In [20]:
response = requests.post(all_urls[0])
soup = BeautifulSoup(response.text, 'html.parser')
soup


<html>
<head>
<title>答申本文</title>
<link href="/images/favicon.ico;jsessionid=57C7B17A91D167FCE2601D3227D67D2E" rel="shortcut icon" type="image/x-icon"/>
</head>
<body>
<div style="font-family:'ＭＳ ゴシック';">
　    <table>
<tbody>
<tr align="left" valign="top">
<td nowrap="nowrap">諮問庁</td>
<td>：</td>
<td>法務大臣</td>
</tr>
<tr align="left" valign="top">
<td nowrap="nowrap">諮問日</td>
<td>：</td>
<td>令和６年１０月２９日（令和６年（行情）諮問第１１７６号）</td>
</tr>
<tr align="left" valign="top">
<td nowrap="nowrap">答申日</td>
<td>：</td>
<td>令和７年７月４日（令和７年度（行情）答申第１６９号）</td>
</tr>
<tr align="left" valign="top">
<td nowrap="nowrap">事件名</td>
<td>：</td>
<td>特定期間に特定法人について弁護士法の規定に基づいて受領した通知の不開示決定（存否応答拒否）に関する件</td>
</tr>
</tbody>
</table>
<p style="font-family:'ＭＳ ゴシック';font-size:19pt;font-weight:bold;text-align:center;">答　申　書</p>
<br/>
<!--1//-->
<p style="font-family:'ＭＳ ゴシック';text-indent:0.0pt;text-align:left;margin:0;margin-left:14.0pt;">第１　審査会の結論</p>
<p style="font-family:'ＭＳ ゴシック';text-indent:14.0pt;text-align:left;margin:0 0 

In [6]:
soup = BeautifulSoup(response.text, 'html.parser')
soup


<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<title>答申データベース検索</title>
<link href="/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/stylesheets/common.css" media="screen" rel="stylesheet"/>
<link href="/bootstrap/css/bootstrap.css" media="screen" rel="stylesheet"/>
<link href="/bootstrap/css/bootstrap.form.css" media="screen" rel="stylesheet"/>
<link href="/treeview/jquery.treeview.css" media="screen" rel="stylesheet"/>
<script src="/javascripts/common.js" type="text/javascript"></script>
<script src="/javascripts/jquery-1.8.0.min.js" type="text/javascript"></script>
<script src="/javascripts/jquery.cookie.js" type="text/javascript"></script>
<script src="/bootstrap/js/bootstrap-alert.js" type="text/javascript"></script>
<script src="/treeview/jquery.treeview.js" type="text/javascript"></script>
<script src="/treeview/jquery.treeview.async.js" type="text/javascript"></script>
</head>
<body data-spy="scroll">
<div class="

In [4]:
# %% [markdown]
# # 総務省答申データベース検索 - 60件取得版

# %%
import requests
from bs4 import BeautifulSoup
import re
import csv

# %%
# セッションの作成とヘッダー設定
session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Referer': 'https://koukai-hogo-db.soumu.go.jp/report/'
}

# %%
# 検索ページにアクセス
print("検索ページにアクセス...")
session.get('https://koukai-hogo-db.soumu.go.jp/report/', headers=headers)

# %%
# フォームデータ（4つすべてチェック）
form_data = {
    'gyouseiKoukai': 'on',    # 行政機関／情報公開法
    'gyouseiHogo': 'on',      # 行政機関／個人情報保護法  
    'dokuhouKoukai': 'on',     # 独立行政法人等／情報公開法
    'dokuhouHogo': 'on',       # 独立行政法人等／個人情報保護法
    'searchType': '1',
    'searchQuery': '',
    'order1': '12',
    'limitOption': '1',
    'openFlag': '1',
    'status': ''
}

# %%
# 初回検索実行
print("検索を実行...")
search_url = 'https://koukai-hogo-db.soumu.go.jp/report/search'
response = session.post(search_url, data=form_data, headers=headers, allow_redirects=True)
print(f"ステータスコード: {response.status_code}")

# %% [markdown]
# ## 検索結果から「本文表示」のURLを取得（3ページ分・60件）

# %%
def extract_report_info(html_content):
    """HTMLから本文表示のURLと詳細情報を抽出"""
    soup = BeautifulSoup(html_content, 'html.parser')
    results = []
    
    # 検索結果テーブルの各行を処理
    rows = soup.find_all('tr')
    
    current_record = {}
    for row in rows:
        # 本文表示リンクを含む行を探す
        link = row.find('a', string=re.compile(r'本文表示'))
        if link:
            # URLを取得
            href = link.get('href')
            if href:
                if href.startswith('/'):
                    url = f"https://koukai-hogo-db.soumu.go.jp{href}"
                else:
                    url = href
                current_record['本文URL'] = url
        
        # 各セルのデータを取得
        cells = row.find_all('td')
        for i, cell in enumerate(cells):
            text = cell.get_text().strip()
            
            # ラベルと値のペアを探す
            if '答申日' in text and i + 1 < len(cells):
                current_record['答申日'] = cells[i + 1].get_text().strip()
            elif '諮問日' in text and i + 1 < len(cells):
                current_record['諮問日'] = cells[i + 1].get_text().strip()
            elif '諮問庁' in text and i + 1 < len(cells):
                current_record['諮問庁'] = cells[i + 1].get_text().strip()
            elif '事件名' in text and i + 1 < len(cells):
                current_record['事件名'] = cells[i + 1].get_text().strip()
        
        # レコードが完成したら追加
        if '本文URL' in current_record and current_record not in results:
            results.append(current_record.copy())
            current_record = {}
    
    return results

# %%
# 3ページ分のデータを取得
all_results = []
all_urls = []

for page_num in range(1, 4):  # 1, 2, 3ページ
    print(f"\n{page_num}ページ目を取得中...")
    
    if page_num == 1:
        # 1ページ目は既に取得済みのresponseを使用
        page_html = response.text
    else:
        # 2ページ目以降は新しくリクエスト
        page_url = f"https://koukai-hogo-db.soumu.go.jp/report/search/page/{page_num}"
        page_response = session.get(page_url, headers=headers)
        
        if page_response.status_code != 200:
            print(f"  エラー: ステータスコード {page_response.status_code}")
            continue
        page_html = page_response.text
    
    # 本文表示リンクを抽出（シンプル版）
    soup = BeautifulSoup(page_html, 'html.parser')
    links = soup.find_all('a', string=re.compile(r'本文表示'))
    
    page_urls = []
    for link in links:
        href = link.get('href')
        if href:
            if href.startswith('/'):
                url = f"https://koukai-hogo-db.soumu.go.jp{href}"
            else:
                url = href
            if url not in page_urls:  # 重複を避ける
                page_urls.append(url)
                all_urls.append(url)
    
    print(f"  {len(page_urls)}件のURLを取得")
    
    # 詳細情報も取得
    page_results = extract_report_info(page_html)
    all_results.extend(page_results)
    print(f"  {len(page_results)}件の詳細情報を取得")

# %%
print(f"\n=== 取得完了 ===")
print(f"合計 {len(all_urls)} 件の本文表示URLを取得しました")
print(f"合計 {len(all_results)} 件の詳細情報を取得しました")

# %%
# 最初の5件のURLを表示
print("\n取得したURLの例（最初の5件）:")
for i, url in enumerate(all_urls[:5], 1):
    print(f"{i}. {url}")

# %%
# 重複チェック
unique_urls = list(set(all_urls))
print(f"\n重複除去後: {len(unique_urls)} 件のユニークなURL")
if len(all_urls) != len(unique_urls):
    print(f"重複があります: {len(all_urls) - len(unique_urls)} 件")

# %%
# CSVファイルに保存
if all_results:
    csv_filename = 'soumu_search_results_60.csv'
    fieldnames = ['答申日', '諮問日', '諮問庁', '事件名', '本文URL']
    
    with open(csv_filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, extrasaction='ignore')
        writer.writeheader()
        writer.writerows(all_results)
    
    print(f"\n詳細情報を '{csv_filename}' に保存しました")
    print(f"保存件数: {len(all_results)}件")

# %%
# URLリストをテキストファイルに保存
with open('report_urls_60.txt', 'w', encoding='utf-8') as f:
    for url in unique_urls:
        f.write(url + '\n')

print(f"\nURLリストを 'report_urls_60.txt' に保存しました")
print(f"保存URL数: {len(unique_urls)}件")

# %%
# ページごとの取得状況を確認
print("\n=== ページごとの取得状況 ===")
for i in range(0, len(all_urls), 20):
    page_num = i // 20 + 1
    page_urls = all_urls[i:i+20]
    print(f"{page_num}ページ目: {len(page_urls)}件")
    if page_urls:
        print(f"  最初のURL: {page_urls[0].split('/')[-1]}")
        print(f"  最後のURL: {page_urls[-1].split('/')[-1]}")

検索ページにアクセス...
検索を実行...
ステータスコード: 200

1ページ目を取得中...
  17件のURLを取得
  20件の詳細情報を取得

2ページ目を取得中...
  11件のURLを取得
  20件の詳細情報を取得

3ページ目を取得中...
  13件のURLを取得
  20件の詳細情報を取得

=== 取得完了 ===
合計 41 件の本文表示URLを取得しました
合計 60 件の詳細情報を取得しました

取得したURLの例（最初の5件）:
1. https://koukai-hogo-db.soumu.go.jp/reportBody/19172
2. https://koukai-hogo-db.soumu.go.jp/reportBody/19166
3. https://koukai-hogo-db.soumu.go.jp/reportBody/19167
4. https://koukai-hogo-db.soumu.go.jp/reportBody/19168
5. https://koukai-hogo-db.soumu.go.jp/reportBody/19169

重複除去後: 33 件のユニークなURL
重複があります: 8 件

詳細情報を 'soumu_search_results_60.csv' に保存しました
保存件数: 60件

URLリストを 'report_urls_60.txt' に保存しました
保存URL数: 33件

=== ページごとの取得状況 ===
1ページ目: 20件
  最初のURL: 19172
  最後のURL: 19160
2ページ目: 20件
  最初のURL: 19159
  最後のURL: 19142
3ページ目: 1件
  最初のURL: 19141
  最後のURL: 19141


In [7]:
# %% [markdown]
# # 答申本文から情報を抽出（シンプル版）

# %%
import requests
from bs4 import BeautifulSoup
import re
import csv
import time
import json

# %%
# セッションとヘッダーの設定
session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Referer': 'https://koukai-hogo-db.soumu.go.jp/report/'
}

# %%
# サンプルURLリスト（最初の10件）
sample_urls = [
    'https://koukai-hogo-db.soumu.go.jp/reportBody/19172',
    'https://koukai-hogo-db.soumu.go.jp/reportBody/19166',
    'https://koukai-hogo-db.soumu.go.jp/reportBody/19167',
    'https://koukai-hogo-db.soumu.go.jp/reportBody/19168',
    'https://koukai-hogo-db.soumu.go.jp/reportBody/19169',
    'https://koukai-hogo-db.soumu.go.jp/reportBody/19170',
    'https://koukai-hogo-db.soumu.go.jp/reportBody/19171',
    'https://koukai-hogo-db.soumu.go.jp/reportBody/19165',
    'https://koukai-hogo-db.soumu.go.jp/reportBody/19163',
    'https://koukai-hogo-db.soumu.go.jp/reportBody/19164'
]

# %%
def extract_toshin_info(html_content, url):
    """
    HTMLから答申情報を抽出（シンプル版）
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 結果を格納
    result = {
        'URL': url,
        '諮問庁': '',
        '諮問日': '',
        '諮問番号': '',
        '答申日': '',
        '答申番号': '',
        '事件名': '',
        '第１_審査会の結論': '',
        '第２_審査請求人の主張の要旨': '',
        '第３_諮問庁の説明の要旨': '',
        '第４_調査審議の経過': '',
        '第５_審査会の判断の理由': '',
        '委員': '',
        '別紙': ''
    }
    
    # ヘッダー情報の抽出（テーブルから）
    header_table = soup.find('table')
    if header_table:
        rows = header_table.find_all('tr')
        for row in rows:
            cells = row.find_all('td')
            if len(cells) >= 3:
                label = cells[0].get_text(strip=True)
                value = cells[2].get_text(strip=True)
                
                if '諮問庁' in label:
                    result['諮問庁'] = value
                elif '諮問日' in label:
                    result['諮問日'] = value
                    # 諮問番号を抽出
                    match = re.search(r'（([^）]+)）', value)
                    if match:
                        result['諮問番号'] = match.group(1)
                elif '答申日' in label:
                    result['答申日'] = value
                    # 答申番号を抽出
                    match = re.search(r'（([^）]+)）', value)
                    if match:
                        result['答申番号'] = match.group(1)
                elif '事件名' in label:
                    result['事件名'] = value
    
    # 本文の全テキストを取得
    full_text = soup.get_text()
    
    # 各セクションを抽出（正規表現でセクション間のテキストを取得）
    sections = {
        '第１_審査会の結論': r'第１\s*審査会の結論(.*?)(?=第２|$)',
        '第２_審査請求人の主張の要旨': r'第２\s*審査請求人の主張の要旨(.*?)(?=第３|$)',
        '第３_諮問庁の説明の要旨': r'第３\s*諮問庁の説明の要旨(.*?)(?=第４|$)',
        '第４_調査審議の経過': r'第４\s*調査審議の経過(.*?)(?=第５|$)',
        '第５_審査会の判断の理由': r'第５\s*審査会の判断の理由(.*?)(?=（第\d部会）|委員|別紙|$)'
    }
    
    for key, pattern in sections.items():
        match = re.search(pattern, full_text, re.DOTALL)
        if match:
            # テキストをクリーンアップ
            text = match.group(1).strip()
            # 過度な空白や改行を整理
            text = re.sub(r'\s+', ' ', text)
            # 最初の500文字まで保存
            result[key] = text[:500]
    
    # 委員名の抽出
    committee_match = re.search(r'委員\s+([^、]+)、委員\s+([^、]+)、委員\s+([^\s]+)', full_text)
    if committee_match:
        result['委員'] = '、'.join(committee_match.groups())
    
    # 別紙の抽出
    bessi_match = re.search(r'別紙(.*?)$', full_text, re.DOTALL)
    if bessi_match:
        bessi_text = bessi_match.group(1).strip()
        bessi_text = re.sub(r'\s+', ' ', bessi_text)
        result['別紙'] = bessi_text[:500]
    
    return result

# %%
# 10件の答申本文を取得して情報抽出
extracted_data = []
failed_urls = []

print("答申本文の取得と情報抽出を開始...")
for i, url in enumerate(sample_urls, 1):
    print(f"\n{i}/10: {url} を処理中...")
    
    try:
        # アクセス間隔を設ける
        if i > 1:
            time.sleep(1)
        
        # ページを取得
        response = session.get(url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            # 情報を抽出
            info = extract_toshin_info(response.text, url)
            extracted_data.append(info)
            
            # 主要情報を表示
            print(f"  諮問庁: {info['諮問庁']}")
            print(f"  答申番号: {info['答申番号']}")
            print(f"  事件名: {info['事件名'][:50]}..." if len(info['事件名']) > 50 else f"  事件名: {info['事件名']}")
        else:
            print(f"  エラー: ステータスコード {response.status_code}")
            failed_urls.append(url)
            
    except Exception as e:
        print(f"  エラー: {str(e)}")
        failed_urls.append(url)

# %%
print(f"\n=== 処理完了 ===")
print(f"成功: {len(extracted_data)}件")
print(f"失敗: {len(failed_urls)}件")

# %%
# CSVファイルに保存
if extracted_data:
    csv_filename = 'toshin_data.csv'
    
    # CSVのフィールド名
    fieldnames = [
        'URL', '諮問庁', '諮問日', '諮問番号', '答申日', '答申番号', '事件名',
        '第１_審査会の結論', '第２_審査請求人の主張の要旨', '第３_諮問庁の説明の要旨',
        '第４_調査審議の経過', '第５_審査会の判断の理由', '委員', '別紙'
    ]
    
    with open(csv_filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(extracted_data)
    
    print(f"\nCSVファイルを '{csv_filename}' に保存しました")

# %%
# JSONファイルにも保存
json_filename = 'toshin_data.json'
with open(json_filename, 'w', encoding='utf-8') as f:
    json.dump(extracted_data, f, ensure_ascii=False, indent=2)

print(f"JSONファイルを '{json_filename}' に保存しました")

# %%
# 最初の3件の内容を確認
print("\n=== 最初の3件の内容確認 ===")
for i, data in enumerate(extracted_data[:3], 1):
    print(f"\n--- {i}件目 ---")
    print(f"URL: {data['URL']}")
    print(f"諮問庁: {data['諮問庁']}")
    print(f"諮問番号: {data['諮問番号']}")
    print(f"答申番号: {data['答申番号']}")
    print(f"事件名: {data['事件名']}")
    print(f"委員: {data['委員']}")
    print(f"第１の冒頭: {data['第１_審査会の結論'][:100]}...")

# %%
# 諮問庁別の集計
from collections import Counter

shimon_cho_count = Counter([d['諮問庁'] for d in extracted_data if d['諮問庁']])
print("\n=== 諮問庁別件数 ===")
for cho, count in shimon_cho_count.most_common():
    print(f"{cho}: {count}件")

# %%
# 抽出できた項目の確認
print("\n=== 抽出状況の確認 ===")
for i, data in enumerate(extracted_data, 1):
    filled_count = sum(1 for v in data.values() if v and v != 'URL')
    total_count = len(data) - 1  # URL以外
    print(f"{i}. {data['答申番号'] or 'N/A'}: {filled_count}/{total_count} 項目を抽出")

# %%
# 失敗したURLの確認
if failed_urls:
    print(f"\n=== 失敗したURL ({len(failed_urls)}件) ===")
    for url in failed_urls:
        print(url)

答申本文の取得と情報抽出を開始...

1/10: https://koukai-hogo-db.soumu.go.jp/reportBody/19172 を処理中...
  諮問庁: 防衛大臣
  答申番号: 令和７年度（行個
  事件名: 本人の退職に係る文書の開示決定に関する件（保有個人情報の特定）

2/10: https://koukai-hogo-db.soumu.go.jp/reportBody/19166 を処理中...
  諮問庁: 防衛大臣
  答申番号: 令和７年度（行情
  事件名: 海上自衛隊が実施する「情報に関連する諸活動（ＩＲＣ）」及び当該文書をつづっている行政文書ファイルにつ...

3/10: https://koukai-hogo-db.soumu.go.jp/reportBody/19167 を処理中...
  諮問庁: 防衛大臣
  答申番号: 令和７年度（行情
  事件名: 基礎情報隊が作成した情報資料の一部開示決定に関する件

4/10: https://koukai-hogo-db.soumu.go.jp/reportBody/19168 を処理中...
  諮問庁: 防衛大臣
  答申番号: 令和７年度（行情
  事件名: 「幹部学校研究瓦版」に該当する文書のうち特定期間において作成されたものの一部開示決定に関する件

5/10: https://koukai-hogo-db.soumu.go.jp/reportBody/19169 を処理中...
  諮問庁: 法務大臣
  答申番号: 令和７年度（行情
  事件名: 特定刑事施設における受刑者の自殺事故報告書の不開示決定（不存在）に関する件

6/10: https://koukai-hogo-db.soumu.go.jp/reportBody/19170 を処理中...
  諮問庁: 法務大臣
  答申番号: 令和７年度（行情
  事件名: 特定期間に特定法人について弁護士法の規定に基づいて受領した通知の不開示決定（存否応答拒否）に関する件

7/10: https://koukai-hogo-db.soumu.go.jp/reportBody/19171 を処理中...
  諮問庁: 防衛大臣
  答申番号: 令和７年度（行情
  事件名: 「この通達の実施に関